![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBERTaForMultipleChoice.ipynb)

## Import ONNX RoBERTaForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `RoBertaForMultipleChoice` or `TFRobertaForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [RoBertaForMultipleChoice](https://huggingface.co/docs/transformers/en/model_doc/roberta#transformers.RobertaForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [https://huggingface.co/SyedArsal/roberta-urdu-small-finetuned-news](https://huggingface.co/SyedArsal/roberta-urdu-small-finetuned-news) model from HuggingFace as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import RobertaTokenizer
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "SyedArsal/roberta-urdu-small-finetuned-news"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

('onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/tokenizer_config.json',
 'onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/special_tokens_map.json',
 'onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/vocab.json',
 'onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/merges.txt',
 'onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -lh {EXPORT_PATH}

total 484M
-rw-r--r-- 1 root root  744 Jun 26 05:20 config.json
-rw-r--r-- 1 root root 1.1M Jun 26 05:20 merges.txt
-rw-r--r-- 1 root root 481M Jun 26 05:20 model.onnx
-rw-r--r-- 1 root root  957 Jun 26 05:20 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Jun 26 05:20 tokenizer_config.json
-rw-r--r-- 1 root root 1.7M Jun 26 05:20 vocab.json


- We need to convert `vocab.json` to `vocab.txt` by extracting and sorting the tokens (one per line), and make sure `merges.txt` is also present.
- Move both `vocab.txt` and `merges.tx`t into an `assets/` folder inside your export path, Spark NLP looks for tokenizer files there when loading ONNX models.

In [4]:
import json

with open(f"{EXPORT_PATH}/vocab.json", "r") as f:
    vocab = json.load(f)

with open(f"{EXPORT_PATH}/vocab.txt", "w") as f:
    for token in vocab.keys():
        f.write(f"{token}\n")

!mkdir -p {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/
!mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

In [5]:
!ls -lR {EXPORT_PATH}/assets

onnx_models/SyedArsal/roberta-urdu-small-finetuned-news/assets:
total 2200
-rw-r--r-- 1 root root 1150119 Jun 26 05:20 merges.txt
-rw-r--r-- 1 root root 1099068 Jun 26 05:21 vocab.txt


Voila! We have our vocab.txt inside assets directory

## Import and Save RobertaForMultipleChoice in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [1]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 28.6 MB/s eta 0:00:00


In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `RoBertaForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
from sparknlp.annotator import RoBertaForMultipleChoice

roBertaMultpleChoiceClassifier = RoBertaForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [5]:
roBertaMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [6]:
!rm -rf {EXPORT_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `RoBertaForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [7]:
data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
     "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?",
     "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])

data.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [8]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import RoBertaForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

roberta_model = RoBertaForMultipleChoice() \
    .load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    roberta_model
])

model = pipeline.fit(data)
results = model.transform(data)

results.selectExpr("question", "choices", "answer.result[0] as predicted_answer").show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------+
|question                                                                                  |choices                                                                 |predicted_answer                    |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.| It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |Germany                